In [1]:
import os
import requests
from datetime import datetime

def descargar_rebill(year: int, month: int, carpeta_destino: str = r"..\data\WR_Suivi_Rebill"):
    """
    Descarga el archivo CSV de suivi rebill para el año y mes especificados.

    Args:
        year (int): Año en formato YYYY (ejemplo: 2025)
        month (int): Mes en formato MM (ejemplo: 7 o 07)
        carpeta_destino (str): Carpeta donde guardar el CSV

    Returns:
        str: Ruta completa del archivo descargado
    """
    # Asegurar formato 2 dígitos para el mes
    mes_str = f"{month:02d}"
    
    # Construir la URL
    base_url = "https://billing.izoswap.fr/admin/export/suivi_rebill/ALL"
    token = "REMOVED_TOKEN"  # tu token fijo
    url = f"{base_url}/{year}-{mes_str}/{token}/csv"

    # Crear carpeta si no existe
    os.makedirs(carpeta_destino, exist_ok=True)

    # Definir nombre del archivo
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_archivo = f"ALL_WR_suivi_rebill_{year}_{mes_str}_{timestamp}.csv"
    ruta_archivo = os.path.join(carpeta_destino, nombre_archivo)

    # Descargar archivo
    response = requests.get(url)
    if response.status_code == 200:
        with open(ruta_archivo, "wb") as f:
            f.write(response.content)
        print(f"✅ Archivo descargado en: {ruta_archivo}")
    else:
        raise Exception(f"❌ Error {response.status_code} al descargar el archivo")

    return ruta_archivo


In [2]:
descargar_rebill(2025, 8)

✅ Archivo descargado en: ..\data\WR_Suivi_Rebill\ALL_WR_suivi_rebill_2025_08_20250818_150217.csv


'..\\data\\WR_Suivi_Rebill\\ALL_WR_suivi_rebill_2025_08_20250818_150217.csv'

In [3]:
import pandas as pd 
file_path=descargar_rebill(2025, 8)
file_name=os.path.basename(file_path)
df=pd.read_csv(file_path,sep=";")
display(df.shape)
display(df.head())

✅ Archivo descargado en: ..\data\WR_Suivi_Rebill\ALL_WR_suivi_rebill_2025_08_20250818_150224.csv


(18859, 44)

,Name,Role,Mail,Score,ID Contract,Periodicity,ID EXT,Subscription,Society,PSP,...,1st attempt,is3ds,isPreabo,isFirstRebillSuccess,hasSuspendedPayment,Debt from period,isManual,isInactive,Last Conversion to Inactive,Last Conversion to Active
0,KitchenTrotterCollection,Error Subscription,patrice.mesle@hotmail.com,9,KTC_377469,1,BXSGDCDGZXSQMHG3,49.9,WAVEROCK,AdyenS2s,...,0,0,0,0,0,NaN,0,0,NaN,NaN
1,KitchenTrotterCollection,Error Subscription,patrice.mesle@hotmail.com,9,KTC_377469,1,QPHPJ6WPPLLWHVG6,49.9,WAVEROCK,AdyenS2s,...,0,0,0,0,0,NaN,0,0,NaN,NaN
2,KitchenTrotterCollection,Error Subscription,patrice.mesle@hotmail.com,9,KTC_377469,1,X4G5G6GTWQM2PGQ9,49.9,WAVEROCK,AdyenS2s,...,0,0,0,0,0,NaN,0,0,NaN,NaN
3,KitchenTrotterCollection,Error Subscription,patrice.mesle@hotmail.com,9,KTC_377469,1,TTWLN69XTRHH9RG6,49.9,WAVEROCK,AdyenS2s,...,0,0,0,0,0,NaN,0,0,NaN,NaN
4,KitchenTrotterCollection,Error Subscription,patrice.mesle@hotmail.com,9,KTC_377469,1,T2PHM4CBXLVFTQG6,49.9,WAVEROCK,AdyenS2s,...,0,0,0,0,0,NaN,0,0,NaN,NaN


In [6]:
import pandas as pd
import numpy as np
from typing import Tuple, Optional, Iterable

def process_rebill(
    df: pd.DataFrame,
    partner_code: str = "118",
    mail_exclude_substring: str = "test",
    column_order: Optional[Iterable[str]] = None,
    strict_columns: bool = False,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Procesa el DataFrame de rebill con los siguientes pasos:
      1) Excluye filas cuyo 'Mail' contenga `mail_exclude_substring` (case-insensitive).
      2) Crea la columna 'UID' con el texto después del '_' en 'ID Contract'.
      3) Crea df_captured: contratos con Partner Code == `partner_code`, 
         manteniendo solo 'ID Contract' y eliminando duplicados.
      4) Crea 'Indicator': si 'ID Contract' ∈ df_captured → usa 'isFirstRebillSuccess',
         en caso contrario → 'Abo Unique'.
      5) Reordena columnas según `column_order`. 
         - Si `strict_columns=True`, lanza error si falta alguna columna.
         - Si `strict_columns=False`, solo mantiene las que existan.

    Parámetros
    ----------
    df : pd.DataFrame
        DataFrame de entrada.
    partner_code : str, opcional (default "118")
        Código del partner para capturar contratos.
    mail_exclude_substring : str, opcional (default "test")
        Substring que, si aparece en 'Mail', excluye la fila.
    column_order : Iterable[str] | None, opcional
        Orden deseado de columnas. Si None, usa un orden por defecto.
    strict_columns : bool, opcional
        Si True, exige que todas las columnas del orden existan (KeyError si faltan).

    Retorna
    -------
    (df_filtered, df_captured) : Tuple[pd.DataFrame, pd.DataFrame]
        df_filtered: DataFrame final, con 'Indicator' y columnas reordenadas.
        df_captured: DataFrame con una columna 'ID Contract', sin duplicados.
    """

    # --- 0) Orden por defecto si no se pasa uno
    default_column_order = [
        "Name", "Role", "Mail", "Score", "ID Contract", "UID", "Periodicity",
        "ID EXT", "Subscription", "Society", "PSP", "MID", "BIN", "BRAND",
        "COUNTRY", "BANK", "Date of Subscription", "Requested Unsub date",
        "Date Unsubscribe", "Date of Transaction", "Amount", "Currency",
        "Payment Status", "Partner Code", "Partner Message", "Abo Unique",
        "Is Ghost", "nb_paid", "nb_fail", "nb_paid_current", "nb_fail_current",
        "original_acquire_mid", "Indicator", "isFirstRebillSuccess",
        "hasSuspendedPayment", "isManual", "isInactive",
        "Last Conversion to Inactive", "Last Conversion to Active"
    ]
    if column_order is None:
        column_order = default_column_order

    # --- 1) Filtrado por Mail no contenga mail_exclude_substring
    if "Mail" not in df.columns:
        raise KeyError("La columna 'Mail' no existe en el DataFrame.")

    pattern = (
        "|".join(map(str, mail_exclude_substring)) 
        if isinstance(mail_exclude_substring, (list, tuple, set)) 
        else str(mail_exclude_substring)
    )
    df_filtered = df[~df["Mail"].astype(str).str.contains(pattern, case=False, na=False)].copy()


    # --- 2) Crear UID desde 'ID Contract' (texto después de '_')
    if "ID Contract" not in df_filtered.columns:
        raise KeyError("La columna 'ID Contract' no existe en el DataFrame.")
    df_filtered["UID"] = df_filtered["ID Contract"].astype(str).str.extract(r'_(.*)$')

    # --- 3) df_captured: contratos con 'Partner Code' == partner_code
    if "Partner Code" not in df_filtered.columns:
        raise KeyError("La columna 'Partner Code' no existe en el DataFrame.")
    df_captured = (
        df_filtered[df_filtered["Partner Code"].astype(str) == str(partner_code)]
        .copy()[["ID Contract"]]
        .drop_duplicates()
        .reset_index(drop=True)
    )

    # --- 4) Crear 'Indicator' en función de pertenencia
    captured_set = set(df_captured["ID Contract"].astype(str))
    mask_in_captured = df_filtered["ID Contract"].astype(str).isin(captured_set)

    for required_col in ["isFirstRebillSuccess", "Abo Unique"]:
        if required_col not in df_filtered.columns:
            raise KeyError(f"La columna '{required_col}' no existe en el DataFrame.")

    df_filtered["Indicator"] = np.where(
        mask_in_captured,
        df_filtered["isFirstRebillSuccess"],
        df_filtered["Abo Unique"]
    )

    # --- 5) Reordenar columnas
    if strict_columns:
        # Exigir todas las columnas
        missing = [c for c in column_order if c not in df_filtered.columns]
        if missing:
            raise KeyError(f"Faltan columnas para el reordenamiento estricto: {missing}")
        df_filtered = df_filtered[list(column_order)]
    else:
        # Tolerante: solo conserva las que existan
        df_filtered = df_filtered[[c for c in column_order if c in df_filtered.columns]]

    return df_filtered, df_captured


In [7]:
df_final, df_captured = process_rebill(
    df,
    partner_code="118",
    mail_exclude_substring=["test", "eviano"],
    strict_columns=False  # pon True si quieres que falle si falta alguna columna
)

display(df_final.shape)
display(df_captured.shape)
display(df_final.head())

file_name = os.path.basename(file_path)
df_final.to_csv(fr"..\data\WR_Suivi_Rebill\OK_{file_name}", index=False, encoding="utf-8")


(18819, 39)

(1734, 1)

,Name,Role,Mail,Score,ID Contract,UID,Periodicity,ID EXT,Subscription,Society,...,nb_paid_current,nb_fail_current,original_acquire_mid,Indicator,isFirstRebillSuccess,hasSuspendedPayment,isManual,isInactive,Last Conversion to Inactive,Last Conversion to Active
0,KitchenTrotterCollection,Error Subscription,patrice.mesle@hotmail.com,9,KTC_377469,377469,1,BXSGDCDGZXSQMHG3,49.9,WAVEROCK,...,0,6,Worldpay Wave Rock,1,0,0,0,0,NaN,NaN
1,KitchenTrotterCollection,Error Subscription,patrice.mesle@hotmail.com,9,KTC_377469,377469,1,QPHPJ6WPPLLWHVG6,49.9,WAVEROCK,...,0,6,Worldpay Wave Rock,0,0,0,0,0,NaN,NaN
2,KitchenTrotterCollection,Error Subscription,patrice.mesle@hotmail.com,9,KTC_377469,377469,1,X4G5G6GTWQM2PGQ9,49.9,WAVEROCK,...,0,6,Worldpay Wave Rock,0,0,0,0,0,NaN,NaN
3,KitchenTrotterCollection,Error Subscription,patrice.mesle@hotmail.com,9,KTC_377469,377469,1,TTWLN69XTRHH9RG6,49.9,WAVEROCK,...,0,6,Worldpay Wave Rock,0,0,0,0,0,NaN,NaN
4,KitchenTrotterCollection,Error Subscription,patrice.mesle@hotmail.com,9,KTC_377469,377469,1,T2PHM4CBXLVFTQG6,49.9,WAVEROCK,...,0,6,Worldpay Wave Rock,0,0,0,0,0,NaN,NaN
